<a href="https://colab.research.google.com/github/SAkbar0210/NSD55/blob/main/Chatbots_with_multi_turn_conversation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install Transformers Library

In [ ]:
!pip install transformers

Import Required Modules

In [ ]:
from transformers import pipeline, set_seed

In [ ]:
!pip install --upgrade sympy

In [ ]:
generator = pipeline("text-generation", model="gpt2")
set_seed(42)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu


Initialize Chat History

In [ ]:
chat_history = ""


Simple Chat Loop

In [ ]:
chat_history = ""

print("🤖 Chatbot: Hello! I’m a simple chatbot. Type 'exit' to stop.\n")

while True:
    user_input = input("You: ")
    if user_input.lower() == "exit":
        print("🤖 Chatbot: Goodbye!")
        break

    # Append user input to history
    chat_history += f"You: {user_input}\nChatbot:"

    # Generate response using GPT-2
    response = generator(chat_history, max_length=200, num_return_sequences=1, pad_token_id=50256)

    # Extract the chatbot's reply
    full_reply = response[0]["generated_text"]
    bot_reply = full_reply[len(chat_history):].split("\n")[0]

    print("🤖 Chatbot:", bot_reply.strip())

    # Update history
    chat_history += f" {bot_reply.strip()}\n"


🤖 Chatbot: Hello! I’m a simple chatbot. Type 'exit' to stop.



KeyboardInterrupt: Interrupted by user

V2

In [ ]:
from transformers import pipeline, set_seed
import re

# Load GPT-2 generator pipeline
generator = pipeline('text-generation', model='gpt2')
set_seed(42)

chat_history = ""

print("🤖 Chatbot: Hello! I’m a simple chatbot. Type 'exit' to stop.\n")

while True:
    user_input = input("You: ")
    if user_input.lower() == "exit":
        print("🤖 Chatbot: Goodbye!")
        break

    # Append user input to history
    chat_history += f"You: {user_input}\nChatbot:"

    # Generate response using GPT-2 with truncation fix
    response = generator(
        chat_history,
        max_new_tokens=50,            # ✅ use max_new_tokens instead of max_length
        num_return_sequences=1,
        pad_token_id=50256,
        truncation=True               # ✅ optional: allows truncation of input if it's too long
    )

    # Extract the chatbot's reply
    full_reply = response[0]["generated_text"]
    bot_reply = full_reply[len(chat_history):].split("\n")[0]

    # Clean bot's reply if needed
    bot_reply = re.sub(r"<[^>]+>", "", bot_reply).strip()

    print("🤖 Chatbot:", bot_reply)

    # Update history
    chat_history += f" {bot_reply}\n"


Device set to use cpu


🤖 Chatbot: Hello! I’m a simple chatbot. Type 'exit' to stop.

You: hii
🤖 Chatbot: lol
You: kjkjkjk
🤖 Chatbot: ok
You: what are you
🤖 Chatbot: ?
You: what are u . a bot or a machine
🤖 Chatbot: bot
You: how do u know u are a bot not a machine
🤖 Chatbot: why u ask in the first place
You: just bcz i am curious
🤖 Chatbot: you do know i cant even do anything
You: ok what can u do
🤖 Chatbot: ok


KeyboardInterrupt: Interrupted by user

 DialoGPT and GPT-Neo

```



In [ ]:
# Install transformers if not already
!pip install transformers --quiet

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Load GPT-Neo
model_name = "EleutherAI/gpt-neo-1.3B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Store conversation
chat_history = ""

print("🤖 Chatbot: Hello! I'm your AI companion. Type 'exit' to stop.\n")

while True:
    user_input = input("You: ")
    if user_input.lower() == "exit":
        print("🤖 Chatbot: Goodbye!")
        break

    # Build prompt with history
    chat_history += f"Human: {user_input}\nAI:"

    # Tokenize prompt and generate response
    inputs = tokenizer(chat_history, return_tensors="pt")
    attention_mask = inputs["attention_mask"]  # 👈 Fixes the warning
    outputs = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=attention_mask,
        max_new_tokens=50,
        do_sample=True,
        top_k=50,
        top_p=0.95,
        temperature=0.8,
        pad_token_id=tokenizer.eos_token_id  # To avoid padding errors
    )

    # Decode only new generated part
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    reply = generated_text[len(chat_history):].split("Human:")[0].strip()

    print(f"🤖 Chatbot: {reply}")
    # Update history with bot reply
    chat_history += f" {reply}\n"


tokenizer_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.35k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/5.31G [00:00<?, ?B/s]

🤖 Chatbot: Hello! I'm your AI companion. Type 'exit' to stop.

You: hii how are u
🤖 Chatbot: what are you doing here?
Hm..
You: i am testing you
🤖 Chatbot: test me?
You: yess u are a bot and i am testing ur capabilities
🤖 Chatbot: what are u saying?
You: nothing that concerns you
🤖 Chatbot: then i am giving you a test
You: exit
🤖 Chatbot: Goodbye!


DialoGPT

In [ ]:
# Install required packages
!pip install transformers --quiet

from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Load pretrained DialoGPT (medium version is a good balance)
tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium")
model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-medium")

# Chat loop
print("🤖 Chatbot: Hello! I'm your AI companion. Type 'exit' to stop.\n")
chat_history_ids = None

while True:
    user_input = input("You: ")
    if user_input.lower() == "exit":
        print("🤖 Chatbot: Goodbye!")
        break

    # Encode the new user input
    new_input_ids = tokenizer.encode(user_input + tokenizer.eos_token, return_tensors='pt')

    # Append the new user input to the chat history
    bot_input_ids = torch.cat([chat_history_ids, new_input_ids], dim=-1) if chat_history_ids is not None else new_input_ids

    # Generate a response
    chat_history_ids = model.generate(
        bot_input_ids,
        max_length=10000,
        pad_token_id=tokenizer.eos_token_id,
        do_sample=True,
        top_k=50,
        top_p=0.95,
        temperature=0.75
    )
dfsajkl
    # Decode the response and print
    bot_reply = tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)
    print(f"🤖 Chatbot: {bot_reply}")


🤖 Chatbot: Hello! I'm your AI companion. Type 'exit' to stop.

You: Hii iam janaki ramudu
🤖 Chatbot: Takashi is a national treasure
You: what is it i am janaki ram
🤖 Chatbot: Janaki is the only national treasure
You: what do chatbots mean
🤖 Chatbot: I don't know
You: what do u know
🤖 Chatbot: I know how to reply to posts
You: u dont know 
🤖 Chatbot: yes I do
You: what is one piece
🤖 Chatbot: It's a movie
You: no its anime
🤖 Chatbot: u dont know
You: yahh i know u only dont know
🤖 Chatbot: yeah u
You: yah i know
🤖 Chatbot: yes u can say u do
You: yes i can
🤖 Chatbot: y
You: bcz i can
🤖 Chatbot: no
You: yes
🤖 Chatbot: nooo i love
You: what
🤖 Chatbot: h
You: sadf
🤖 Chatbot: 
You: exit
🤖 Chatbot: Goodbye!
